In [1]:
import sys
import os

import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import pytorch_lightning as pl
import collections


from ai.trainers.time_series import Trainer
from ai.models.model_v1 import Model_v1
from ai.evaluation import Summary
from ai.loaders import DataLoader_v1
from ai.preprocessing import Lag


from torch.utils.data import Dataset
from torch.utils.data import Subset, DataLoader
from typing import List, Dict, Any
from functools import reduce

from sklearn.preprocessing import StandardScaler
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.summarize import WindowSummarizer
from sktime.transformations.series.impute import Imputer
from sktime.transformations.compose import TransformerPipeline
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.transformations.series.dropna import DropNA
from sktime.transformations.series.subset import IndexSubset

sys.path.append(os.path.abspath('..'))
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
%matplotlib inline
    

In [27]:

data_path = os.path.join(os.getenv("AI_DATA_PATH"), "compressor.csv")
   
features = {
        "input_1" : "PH (CBM) 1st Stage Poly Head Dev",
        "input_2" : "PH (CBM) 1st Stage Press Rat Dev",
        "input_3" : "PH (CBM) 1st Stage ActCompr Poly Eff",
        "input_4" : "PH (CBM) 1st Stg ActCompr Poly Head",
        "target"  : "PH (CBM) 1st Stg ActCompr Poly Head",
    }

lags = {
    "input_1" : Lag(5) ,
    "input_2" : Lag(5) ,
    "input_3" : Lag(5) ,
    "input_4" : Lag(5) ,
    "target"  : Lag(-1),
}

preprocessors = {
    "input_1" : TransformerPipeline([("scaler", StandardScaler()) ]),
    "input_2" : TransformerPipeline([("scaler", StandardScaler()) ]),
    "input_3" : TransformerPipeline([("scaler", StandardScaler()) ]),
    "input_4" : TransformerPipeline([("scaler", StandardScaler()) ]),
    "target"  : TransformerPipeline([("scaler", StandardScaler()) ]),
}

input_names = ['input_1','input_2','input_3', 'input_4']
target_name = 'target'

dataset = DataLoader_v1(data_path, features, input_names, 'target', lags, preprocessors)
print(len(dataset))

cv = SlidingWindowSplitter(window_length=20000, step_length=20000)

model = Model_v1(dataset)

trainer = Trainer(model, cv)

trainer.fit(dataset)




2026-02-03 00:24:24.360 | INFO     | ai.trainers.time_series:fit:88 - Starting Training for Fold 0/27
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


547503


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name      ┃ Type       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ criterion │ MSELoss    │      0 │ train │     0 │
│ 1 │ net       │ Sequential │  1.4 K │ train │     0 │
└───┴───────────┴────────────┴────────┴───────┴───────┘

Trainable params: 1.4 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.4 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 6                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/Users/jodafons/Desktop/codes/ai/.ai-env/lib/python3.10/site-packages/rich/live.py:260: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/jodafons/Desktop/codes/ai/.ai-env/lib/python3.10/site-packages/pytorch_lightning/utilities/_pytree.py:21: 
`isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` 
instead.

/Users/jodafons/Desktop/codes/ai/.ai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_con
nector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the 
value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

/Users/jodafons/Desktop/codes/ai/.ai-env/lib/python3.10/site-packages/rich/live.py:260: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/jodafons/Desktop/codes/ai/ai/models/model_v1.py:43: UserWarning: Using a non-tuple sequence for 
multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In
pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or
a different result (Triggered internally at 
/Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/python_variable_indexing.cpp:353.)
  x = torch.cat([x[feature_name] for feature_name in self.input_features], dim=1)

IndexError: too many indices for tensor of dimension 3

In [ ]:


# 3. Cross-Validation Configuration
fold_results = []



for fold, (train_index, val_index) in enumerate(cv.split(dataset.index())):


    dataset.fit(train_index)
    
    # Create Subsets
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    val_dataset   = torch.utils.data.Subset(dataset, val_index)

    train_loader  = DataLoader(train_dataset, batch_size=5, shuffle=False)
    val_loader    = DataLoader(val_dataset, batch_size=5, shuffle=False)

    d = next(iter(train_loader))
    print([dd.shape for dd in d])
